In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/2019top10songs.csv')

In [33]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 8 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace(',','').replace('!','')
artist2 = artist.lower().replace('\'','').replace(',','').replace(' ','-').replace('!','')
artistl = [artist1, artist2, artist1[:artist1.rfind('and')], artist2[:artist2.rfind('-and')], artist1[:artist1.rfind('featuring')], artist2[:artist2.rfind('featuring')], artist1[:artist1.rfind(',')], artist1[artist1.find(','):artist1.rfind('and')], artist1[artist1.rfind('and')+3:]]
# The first three entries of the above list are standard single artist titles. The next three handle cases of collaborations between two artists. The next four after that handle collaborations between three artists.
title = ''.join(songdata['Title'][i].split()).lower().replace('\'','').replace(',','').replace('!','') # azlyrics seems to represent all song titles in the same way.
titlel = [title, 'a' + title, 'the' + title] # However, there are very rare cases, such as 'Holly Jolly Christmas', where azlyrics includes an 'a' or a 'the' at the beginning of a song and Wikipedia does not. These cases will only be tested if all of the artist configurations fail.
for j in range(len(titlel)):
    for k in range(len(artistl)):
        print(url + artistl[k] + '/' + titlel[j] + '.html')
        if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
            html = requests.get(url + artistl[k] + '/' + titlel[j] + '.html',cookies=cookies,headers=headers).text
            soup = BeautifulSoup(html, 'lxml')
            print(url + artistl[k] + '/' + titlel[j] + '.html')
            break
    if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
        break
    if j == len(titlel) - 1:
        print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

https://www.azlyrics.com/lyrics/burlives/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burl-ives/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burlive/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burl-ive/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burlive/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burl-ive/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burlive/hollyjollychristmas.html
https://www.azlyrics.com/lyrics//hollyjollychristmas.html
https://www.azlyrics.com/lyrics/rlives/hollyjollychristmas.html
https://www.azlyrics.com/lyrics/burlives/ahollyjollychristmas.html
https://www.azlyrics.com/lyrics/burlives/ahollyjollychristmas.html


In [22]:
artist1

'lilnasxfeaturingbillyraycyrus1'

In [34]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
try:
    my_text = my_text[:my_text.rindex("[")-1] # Remove miscellaneous notes at the end of lyrics.
except ValueError:
    my_text = my_text
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.

In [35]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(my_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

['ding-dong-d', 'ding-dong-d', 'ding-dong-d', 'have', 'a', 'holli', 'jolli', 'christma', 'ding-dong-d', 'it', "'s", 'the', 'best', 'time', 'of', 'the', 'year', 'i', 'do', "n't", 'know', 'if', 'there', "'ll", 'be', 'snow', 'but', 'have', 'a', 'cup', 'of', 'cheer', 'ding-dong-d', 'have', 'a', 'holli', 'jolli', 'christma', 'ding-dong-d', 'and', 'when', 'you', 'walk', 'down', 'the', 'street', 'say', 'hello', 'to', 'friend', 'you', 'know', 'and', 'everyon', 'you', 'meet', 'oh', 'ho', 'the', 'mistleto', 'hung', 'where', 'you', 'can', 'see', 'somebodi', 'wait', 'for', 'you', 'kiss', 'her', 'onc', 'for', 'me', 'ding-dong-d', 'have', 'a', 'holli', 'jolli', 'christma', 'ding-dong-d', 'and', 'in', 'case', 'you', 'did', "n't", 'hear', 'oh', 'by', 'golli', 'have', 'a', 'holli', 'jolli', 'christma', 'thi', 'year', 'have', 'a', 'holli', 'jolli', 'christma', 'it', "'s", 'the', 'best', 'time', 'of', 'the', 'year', 'have', 'a', 'holli', 'jolli', 'christma', 'and', 'when', 'you', 'walk', 'down', 'the', '

In [36]:
unique_count = len(set(stem_text))
unique_count

60

In [37]:
len(my_text)

178